## Steps to setup connection of Databricks with MYSQL AND BQ
- Reads the config_table from MYSQL Workbench and take only those tables which are having active_flag as 1 and load_flag as 1 
- Export each BQ Table to GCS Stagging Layer
- Update and Log the status of the Load in config_table

In [0]:
dbutils.widgets.text("mysql_host","34.56.24.205")
mysql_host = dbutils.widgets.get("mysql_host")

dbutils.widgets.text("mysql_port","3306")
mysql_port = dbutils.widgets.get("mysql_port")

dbutils.widgets.text("mysql_root","root")
mysql_user = dbutils.widgets.get("mysql_root")

dbutils.widgets.text("mysql_password","Anupamnaina#18")
mysql_password = dbutils.widgets.get("mysql_password")

dbutils.widgets.text("bq_sa_key",'/Volumes/workspace/default/csv/datamigrationproject-483310-739969975183.json')
bq_sa_key = dbutils.widgets.get("bq_sa_key")

In [0]:
print(mysql_host)

In [0]:
%pip install mysql-connector-python google-cloud-bigquery

In [0]:
import json, os, datetime as dt
import mysql.connector as mc
from google.cloud import bigquery
from contextlib import contextmanager 
# contextlib python library help us to setup and clean up the resources automatically

MYSQL = {
    'host':mysql_host.strip(),
    'port':int(mysql_port),
    'user':mysql_user.strip(),
    'pwd':mysql_password
}
BQ_KEY=bq_sa_key.replace('dbfs:/','/dbfs/') 

In [0]:
MYSQL

In [0]:
@contextmanager
def mysql_conn():
    conn = mc.connect(
        host = MYSQL['host'], 
        port = MYSQL['port'],
        user = MYSQL['user'],
        password = MYSQL['pwd']
    )
    try:
        yield conn
    except Exception as e:
        print(f"Failed to connect to MYSQL \nHere is the error -: \n{e}")
    # yield returns a value temporarily and pauses the function, then resumes later from the same place.
    finally:
        conn.close()

In [0]:
def fetch_eligible_tables():
    with mysql_conn() as conn:
        cur = conn.cursor()
        cur.execute("Select * from GCPMigrationMeta.config_table where active_flag=1 and load_flag=1")
        rows = cur.fetchall()
        cur.close()
    return rows

fetch_eligible_tables()

In [0]:
def bq_to_gcs_status(table_name, status, error = None):
    with mysql_conn() as conn:
        cur = conn.cursor()
        if Status = 'IN_PROGRESS':
            cur.execute(f"""
                        UPDATE GCPMigrationMeta.config_table
                        SET bq_to_gcs_status = 'IN_PROGRESS',
                        last_run = NOW(),
                        error_message = NULL
                        WHERE table_name = '{table_name}'
                        """)